In [1]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Callable, Dict, List, Tuple, Any, Optional
from statistics import mean
import numpy as np
from numpy.random import default_rng, Generator

In [ ]:
# ========== 卷轴基础 ==========

@dataclass(frozen=True)
class Scroll:
    name: str
    success_p: float
    atk_gain: int
    destroy_on_fail_p: float = 0.0

SCROLL_A = Scroll("A_10p_+3", success_p=0.11, atk_gain=3, destroy_on_fail_p=0.0)
SCROLL_B = Scroll("B_60p_+2", success_p=0.66, atk_gain=2, destroy_on_fail_p=0.0)
SCROLL_C = Scroll("C_30p_+3_boom50", success_p=0.33, atk_gain=3, destroy_on_fail_p=0.50)
SCROLL_D = Scroll("D_70p_+2_boom50", success_p=0.77, atk_gain=2, destroy_on_fail_p=0.50)

SCROLL_SET = {"A": SCROLL_A, "B": SCROLL_B, "C": SCROLL_C, "D": SCROLL_D}
NAME_TO_KEY = {SCROLL_A.name: "A", SCROLL_B.name: "B", SCROLL_C.name: "C", SCROLL_D.name: "D"}  # 便于反查类型键

# ========== 状态与类型 ==========

@dataclass
class ItemState:
    attempts_used: int = 0               # 砸了几次卷轴
    successes: int = 0                   # 卷轴成功的次数
    total_atk: int = 0                   # 卷轴给装备增加的属性
    destroyed: bool = False              # 装备是否因卷轴消失
    history: List[str] = None            # 保存卷轴 display 名称
    result_history: List[bool] = None    # 整个砸卷过程的成功率

    def __post_init__(self):
        if self.history is None:
            self.history = []
        if self.result_history is None:
            self.result_history = []

@dataclass
class RunStats:
    total_items: int
    final_item_state: ItemState
    item_slots_all: List[int]
    item_attacks_all: List[int]
    item_destroy_all: List[bool]
    total_scrolls_by_type: Dict[str, int]  # 本次试验总计的各类型卷轴消耗（跨多件）

Strategy = Callable[[ItemState, int, int], Scroll]
StopCondition = Callable[[ItemState], bool]

# ========== Monte Carlo ==========

def simulate_until_one_satisfy(
    rng: Generator,
    strategy: Strategy,
    upgrades: int, # 砸几次卷轴
    stop_condition: StopCondition,
) -> RunStats:

    total_items = 0
    destroy_list: List[bool] = []
    all_item_attacks: List[int] = []
    all_item_slot: List[int] = []
    total_scrolls_by_type = {"A": 0, "B": 0, "C": 0, "D": 0}

    while True:
        state = ItemState()

        # 对每件装备的砸卷处理
        while state.attempts_used < upgrades:
            remaining = upgrades - state.attempts_used
            scroll = strategy(state, remaining, item_index=total_items)

            # 记录本次使用的卷轴
            state.history.append(scroll.name)
            # 按类型计数
            key = NAME_TO_KEY[scroll.name]
            total_scrolls_by_type[key] += 1

            # 消耗一次尝试
            state.attempts_used += 1

            # 判定
            if rng.random() < scroll.success_p:
                state.successes += 1
                state.total_atk += scroll.atk_gain
                state.result_history.append(True)
            else:
                state.result_history.append(False)
                break
                # 判定该装备是否消失
                if scroll.destroy_on_fail_p > 0 and (rng.random() < scroll.destroy_on_fail_p):
                    state.destroyed = True
                    break

        # 累计全局统计
        total_items += 1
        destroy_list.append(state.destroyed)
        all_item_attacks.append(state.total_atk)
        all_item_slot.append(state.attempts_used)

        if stop_condition(state):
            return RunStats(
                total_items=total_items,
                final_item_state=state,
                item_slots_all=all_item_slot[:-1],
                item_attacks_all=all_item_attacks[:-1],
                item_destroy_all=destroy_list[:-1],
                total_scrolls_by_type=total_scrolls_by_type,
            )

def monte_carlo_mix(
    rng: Generator,
    trials: int,
    strategy: Strategy,
    upgrades: int, # 砸几次卷
    stop_condition: StopCondition,
) -> Dict[str, Any]:
    
    total_items_list: List[int] = []
    final_atk_list: List[int] = []
    final_successes_list: List[int] = []
    item_slots_all: List[List[int]] = []
    item_attacks_all: List[List[int]] = []
    item_destroy_all: List[List[bool]] = []

    # 每次试验的类型计数
    per_trial_scrolls_by_type: List[Dict[str, int]] = []
    for _ in range(trials):
        stats = simulate_until_one_satisfy(rng, strategy, upgrades, stop_condition)
        total_items_list.append(stats.total_items)
        final_atk_list.append(stats.final_item_state.total_atk)
        final_successes_list.append(stats.final_item_state.successes)
        item_slots_all.append(stats.item_slots_all)
        item_attacks_all.append(stats.item_attacks_all)
        item_destroy_all.append(stats.item_destroy_all)
        per_trial_scrolls_by_type.append(stats.total_scrolls_by_type)

    return {
        # 原有分布（总卷轴/总装备）
        "items_distribution": total_items_list,
        # 新增：按类型的分布与均值
        "scrolls_distribution_by_type": per_trial_scrolls_by_type,  # 每种卷轴的消耗量
        "item_slots_all": item_slots_all, # 所有item的砸卷数
        "item_attacks_all": item_attacks_all, # 所有item的属性强化值
        "item_destroy_all": item_destroy_all, # 所有item的损坏状态
        "final_atk_list": final_atk_list, # 最终item的属性强化值
        "final_successes_list": final_successes_list, # 最终item成功的次数
    }

In [14]:
# ========== 常用目标（StopCondition）示例 ==========
def stop_all_success(upgrades: int) -> StopCondition:
    # 目标：在不爆装情况下，成功次数 == upgrades
    def cond(state: ItemState) -> bool:
        return (state.successes == upgrades) and (not state.destroyed)
    return cond

def stop_atk_at_least(target_atk: int) -> StopCondition:
    # 目标：总攻击力达到阈值（不爆装）
    def cond(state: ItemState) -> bool:
        return (state.total_atk >= target_atk) and (not state.destroyed)
    return cond

def stop_successes_at_least(target_succ: int) -> StopCondition:
    # 目标：成功次数达到阈值（不爆装）
    def cond(state: ItemState) -> bool:
        return (state.successes >= target_succ) and (not state.destroyed)
    return cond

def stop_when_no_slot(num_slots: int) -> StopCondition:
    # 目标：升级次数用完（无论爆不爆）
    def cond(state: ItemState) -> bool:
        return (state.attempts_used == num_slots)
    return cond

In [15]:
# ========== 策略示例（可自定义更复杂规则） ==========

def strategy_fixed_sequence(seq: List[str]) -> Strategy:
    """
    固定序列：按照提供的卷轴名序列循环使用（长度不足时可循环或截断）。
    例如：["C", "C", "B", "B", "B", "B", "B"]
    """
    scrolls = [SCROLL_SET[name] for name in seq]

    def strat(state: ItemState, remaining: int, item_index: int) -> Scroll:
        idx = min(state.attempts_used, len(scrolls) - 1)
        return scrolls[idx]
    return strat

In [16]:
def transform_attack_to_scroll(
    attack_sequence: list,
    attack_five_scroll: str="A",
    attack_two_scroll: str="B",
    ):

    scroll_sequence = []
    for attack_value in attack_sequence:
        if attack_value == 2:
            scroll_sequence.append(attack_two_scroll)
        elif attack_value == 5:
            scroll_sequence.append(attack_five_scroll)
        else:
            raise NotImplementedError
    
    return scroll_sequence

## 14g 手套

In [17]:
WEAPON_SCROLL_PRICE = {"A": 269, "B": 6.4, "C": 4800, "D": 9.5}
GROVE_SCROLL_PRICE = {"A": 33, "B": 597, "C": 14959, "D": 2325}
SHIELD_SCROLL_PRICE = {"A": 79, "B": 2000, "C": 1522, "D": 700}
WEAPON_SWORD = {"108": 25000, "107": 15000}

In [18]:
# rng = default_rng(42)
# upgrades = 7
# trials = 20000

# # 目标：全成功
# target = stop_atk_at_least(14)

# # 策略
# results = {}

# scroll_list = ["D", "D", "D", "D", "D", "D", "D"]
# strat1 = strategy_fixed_sequence(scroll_list)
# res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

# small_dict = [i['D'] for i in res1["scrolls_distribution_by_type"]]
# results["-".join(scroll_list)] = {
#     SCROLL_SET["D"].name: np.mean(small_dict), 
#     "items": res1['items_distribution'],
#     "items_attack": res1['item_attacks_all'],
#     "items_slot": res1['item_slots_all']
# }

# scroll_list = ["B", "B", "B", "B", "B", "B", "B"]
# strat1 = strategy_fixed_sequence(scroll_list)
# res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

# small_dict = [i['B'] for i in res1["scrolls_distribution_by_type"]]
# results["-".join(scroll_list)] = {
#     SCROLL_SET["B"].name: np.mean(small_dict), 
#     "items": res1['items_distribution'],
#     "items_attack": res1['item_attacks_all'],
#     "items_slot": res1['item_slots_all']
# }

In [19]:
sequences = []

for a_pos in range(7):
    base_sequence = ["B", "B", "B", "B", "B", "B", "B"]
    base_sequence[a_pos] = "A"
    sequences.append(base_sequence)

for a_pos in range(7):
    base_sequence = ["D", "D", "D", "D", "D", "D", "D"]
    base_sequence[a_pos] = "A"
    sequences.append(base_sequence)

for c_pos in range(7):
    base_sequence = ["B", "B", "B", "B", "B", "B", "B"]
    base_sequence[c_pos] = "C"
    sequences.append(base_sequence)

for c_pos in range(7):
    base_sequence = ["D", "D", "D", "D", "D", "D", "D"]
    base_sequence[c_pos] = "C"
    sequences.append(base_sequence)

In [20]:
from tqdm import tqdm

rng = default_rng(42)
upgrades = 7
trials = 20000

# 目标：全成功
target = stop_atk_at_least(17)

results = {}
for scroll_list in tqdm(sequences):
    strat1 = strategy_fixed_sequence(scroll_list)
    res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

    results["-".join(scroll_list)] = {
        "scrolls": res1["scrolls_distribution_by_type"], 
        "items": res1['items_distribution'],
        "items_attack": res1['item_attacks_all'],
        "items_slot": res1['item_slots_all']
    }

100%|██████████| 28/28 [01:37<00:00,  3.47s/it]


In [21]:
def get_num_scrolls(scrolls_distribution):
    num_a = []
    num_b = []
    num_c = []
    num_d = []
    for i in scrolls_distribution:
        num_a.append(i['A'])
        num_b.append(i["B"])
        num_c.append(i['C'])
        num_d.append(i['D'])

    return {"A": np.mean(num_a), "B": np.mean(num_b), "C": np.mean(num_c), "D": np.mean(num_d)}

In [ ]:
mxb_to_rmb = 56 # 56W mxb = 1rmb
GROVE_ATTACK_PRICE = {
    "10": 1500,
    "11": 2500,
    "12": 5000,
    "13": 14000,
    "14": 41000,
    "15": 2200*mxb_to_rmb,
    "16": 6000*mxb_to_rmb,
    "17": 12500*mxb_to_rmb
}

# def get_bad_grove_price(item_attacks_all, item_slots_all):
#     sell_item = []
#     for index, attack_list in enumerate(item_attacks_all):
#         attack_array = np.array(attack_list)
#         slots_array = np.array(item_slots_all[index])

#         bad_item_price = []
#         for i,v_i in enumerate(attack_array):
#             if v_i < 10:
#                 continue

#             if (v_i == 10) & (slots_array[i]==6):
#                 bad_item_price.append(GROVE_ATTACK_PRICE["10good"])
#             elif (v_i == 12) & (slots_array[i]==7):
#                 bad_item_price.append(GROVE_ATTACK_PRICE["12"])
#             else:
#                 raise ValueError(f"{slots_array[i]} {v_i}")
                
#         if bad_item_price:
#             sell_item.append(np.mean(bad_item_price))
#     return sell_item

def get_bad_grove_price(item_attacks_all):
    sell_item = []
    for attack_list in item_attacks_all:
        attack_array = np.array(attack_list)
        v,c = np.unique(attack_array[attack_array>=10], return_counts=True)

        bad_item_price = 0
        for i,v_i in enumerate(v):
            if v_i > 15:
                continue
            bad_item_price += np.mean(GROVE_ATTACK_PRICE[str(v_i)]) * c[i]

        sell_item.append(bad_item_price)
    return sell_item

In [30]:
1/0.77**6

4.79794923929466

In [23]:
GROVE_SCROLL_PRICE = {
    "A": 33, 
    "B": 597, 
    "C": 14959, 
    "D": 2325
}
item_price = 2500

first = True
for scroll_list, result_i in results.items():

    total_price = 0
    bad_grove_price = 0
    for key, value in result_i.items():
        if key == "items":
            total_price += np.mean(np.array(result_i["items"]) * item_price)
        elif key == "items_attack":
            bad_grove_price = np.mean(get_bad_grove_price(value))
        elif key == "items_slot":
            continue
        elif key == "scrolls":
            num_scrolls = get_num_scrolls(value)
            for s,v in num_scrolls.items():
                scroll_price = v * GROVE_SCROLL_PRICE[s]
                total_price += scroll_price
        else:
            raise ValueError
    
    bad_grove_sell = bad_grove_price / mxb_to_rmb
    cost_rmb = total_price / mxb_to_rmb
    if first:
        print(f"{scroll_list}: {np.mean(result_i['items']):.2f} items {total_price:.2f} bad grove {bad_grove_price:.2f} total cost {(total_price-bad_grove_price):.2f} {GROVE_SCROLL_PRICE}")
        first = False
    else:
        print(f"{scroll_list}: {np.mean(result_i['items']):.2f} items {total_price:.2f} bad grove {bad_grove_price:.2f} total cost {(total_price-bad_grove_price):.2f}")

A-B-B-B-B-B-B: 109.19 items 295919.34 bad grove 77213.51 total cost 218705.83 {'A': 33, 'B': 597, 'C': 14959, 'D': 2325}
B-A-B-B-B-B-B: 110.12 items 355687.39 bad grove 76833.10 total cost 278854.28
B-B-A-B-B-B-B: 111.03 items 396708.97 bad grove 77475.37 total cost 319233.60
B-B-B-A-B-B-B: 111.78 items 424680.59 bad grove 78831.21 total cost 345849.38
B-B-B-B-A-B-B: 108.25 items 427610.57 bad grove 73530.94 total cost 354079.63
B-B-B-B-B-A-B: 109.83 items 444614.92 bad grove 82414.97 total cost 362199.95
B-B-B-B-B-B-A: 110.33 items 453918.47 bad grove 47637.05 total cost 406281.42
A-D-D-D-D-D-D: 43.65 items 149055.56 bad grove 43332.64 total cost 105722.92
D-A-D-D-D-D-D: 43.99 items 240881.54 bad grove 44341.92 total cost 196539.63
D-D-A-D-D-D-D: 43.49 items 307163.59 bad grove 42776.44 total cost 264387.15
D-D-D-A-D-D-D: 43.14 items 357543.16 bad grove 42791.26 total cost 314751.90
D-D-D-D-A-D-D: 43.22 items 398847.80 bad grove 42014.28 total cost 356833.52
D-D-D-D-D-A-D: 43.46 items

In [4]:
40000 / 56

714.2857142857143

In [21]:
import itertools
scrolls = range(10)
all_items = itertools.product(scrolls, repeat=7)

In [23]:
list(all_items)

[(0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 2),
 (0, 0, 0, 0, 0, 0, 3),
 (0, 0, 0, 0, 0, 0, 4),
 (0, 0, 0, 0, 0, 0, 5),
 (0, 0, 0, 0, 0, 0, 6),
 (0, 0, 0, 0, 0, 0, 7),
 (0, 0, 0, 0, 0, 0, 8),
 (0, 0, 0, 0, 0, 0, 9),
 (0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 1, 2),
 (0, 0, 0, 0, 0, 1, 3),
 (0, 0, 0, 0, 0, 1, 4),
 (0, 0, 0, 0, 0, 1, 5),
 (0, 0, 0, 0, 0, 1, 6),
 (0, 0, 0, 0, 0, 1, 7),
 (0, 0, 0, 0, 0, 1, 8),
 (0, 0, 0, 0, 0, 1, 9),
 (0, 0, 0, 0, 0, 2, 0),
 (0, 0, 0, 0, 0, 2, 1),
 (0, 0, 0, 0, 0, 2, 2),
 (0, 0, 0, 0, 0, 2, 3),
 (0, 0, 0, 0, 0, 2, 4),
 (0, 0, 0, 0, 0, 2, 5),
 (0, 0, 0, 0, 0, 2, 6),
 (0, 0, 0, 0, 0, 2, 7),
 (0, 0, 0, 0, 0, 2, 8),
 (0, 0, 0, 0, 0, 2, 9),
 (0, 0, 0, 0, 0, 3, 0),
 (0, 0, 0, 0, 0, 3, 1),
 (0, 0, 0, 0, 0, 3, 2),
 (0, 0, 0, 0, 0, 3, 3),
 (0, 0, 0, 0, 0, 3, 4),
 (0, 0, 0, 0, 0, 3, 5),
 (0, 0, 0, 0, 0, 3, 6),
 (0, 0, 0, 0, 0, 3, 7),
 (0, 0, 0, 0, 0, 3, 8),
 (0, 0, 0, 0, 0, 3, 9),
 (0, 0, 0, 0, 0, 4, 0),
 (0, 0, 0, 0, 0,

In [16]:
len(list(all_items))

16384

In [24]:
num_scrolls = get_num_scrolls(results["A-D-D-D-D-D-D"]["scrolls"])

In [26]:
0.33*1 + 0.66*0.33*2 + 0.66**2*0.33*3 + 0.66**3*0.33*4 + 0.66**4*0.33*5 + 0.66**5*0.33*6

2.137383714048

In [3]:
0.33 + 0.66*0.33 + 0.66**2*0.33 + 0.66**3*0.33 + 0.66**4*0.33 + 0.66**5*0.33 + 0.66**6*0.33 + 0.66**7

0.9721926943238403

In [25]:
num_scrolls

{'A': 43.6532, 'B': 0.0, 'C': 0.0, 'D': 16.5514}

In [ ]:
from plot import create_figure
import matplotlib.pyplot as plt

key = 'B-B-B-B-B-B-B'
value = results['B-B-B-B-B-B-B']

x_list = value['items']
num_items = []
probability = []
for i in range(0,np.max(x_list)+1):
    probability.append(np.sum(np.array(x_list) < i+1) / len(x_list))
    num_items.append(i+1)

valid_num_items = range(0, int(np.mean(x_list) + 3*np.std(x_list, ddof=1)))
valid_probability = np.array(probability)[valid_num_items]

fig, axs = create_figure(1,3)
fig.suptitle(key)
ax1 = fig.add_subplot(axs[0])
ax1.plot(valid_probability)
ax1.set_title("CDF")

ax2 = fig.add_subplot(axs[1])
ax2.hist(x_list, bins=np.arange(0,np.max(x_list),5))
ax2.axvline(np.max(valid_num_items), color='r')
ax2.set_title("PDF")

ax3 = fig.add_subplot(axs[2])
ax3.plot(np.diff(valid_probability))
ax3.set_title(key)

key = 'D-D-D-D-D-D-D'
value = results['D-D-D-D-D-D-D']

x_list = value['items']
num_items = []
probability = []
for i in range(0,np.max(x_list)+1):
    probability.append(np.sum(np.array(x_list) < i+1) / len(x_list))
    num_items.append(i+1)

valid_num_items = range(0, int(np.mean(x_list) + 3*np.std(x_list, ddof=1)))
valid_probability = np.array(probability)[valid_num_items]


fig, axs = create_figure(1,3)
fig.suptitle(key)
ax1 = fig.add_subplot(axs[0])
ax1.plot(valid_probability)
ax1.set_title("CDF")

ax2 = fig.add_subplot(axs[1])
ax2.hist(x_list, bins=np.arange(0,np.max(x_list),5))
ax2.axvline(np.max(valid_num_items), color='r')
ax2.set_title("PDF")

ax3 = fig.add_subplot(axs[2])
ax3.plot(np.diff(valid_probability))
ax3.set_title(key)

## 1张30%

In [ ]:
rng = default_rng(42)
upgrades = 7
trials = 20000

# 目标：全成功
target = stop_atk_at_least(17)

# 策略
results = {}
for c_pos in range(upgrades):
    scroll_list = ["D", "D", "D", "D", "D", "D", "D"]
    scroll_list[c_pos] = "C"
    strat1 = strategy_fixed_sequence(scroll_list)
    res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

    small_dict = [i['D'] for i in res1["scrolls_distribution_by_type"]]
    big_dist = [i['C'] for i in res1["scrolls_distribution_by_type"]]
    results["-".join(scroll_list)] = {
        SCROLL_SET["D"].name: np.mean(small_dict), 
        SCROLL_SET["C"].name: np.mean(big_dist), 
        "items": res1['items_distribution'],
        "items_attack": res1['item_attacks_all']
    }

for c_pos in range(upgrades):
    scroll_list = ["D", "D", "D", "D", "D", "D", "D"]
    scroll_list[c_pos] = "A"
    strat1 = strategy_fixed_sequence(scroll_list)
    res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

    small_dict = [i['D'] for i in res1["scrolls_distribution_by_type"]]
    big_dist = [i['A'] for i in res1["scrolls_distribution_by_type"]]
    results["-".join(scroll_list)] = {
        SCROLL_SET["D"].name: np.mean(small_dict), 
        SCROLL_SET["A"].name: np.mean(big_dist), 
        "items": res1['items_distribution'],
        "items_attack": res1['item_attacks_all']
    }

for c_pos in range(upgrades):
    scroll_list = ["B", "B", "B", "B", "B", "B", "B"]
    scroll_list[c_pos] = "C"
    strat1 = strategy_fixed_sequence(scroll_list)
    res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

    small_dict = [i['B'] for i in res1["scrolls_distribution_by_type"]]
    big_dist = [i['C'] for i in res1["scrolls_distribution_by_type"]]
    results["-".join(scroll_list)] = {
        SCROLL_SET["B"].name: np.mean(small_dict), 
        SCROLL_SET["C"].name: np.mean(big_dist), 
        "items": res1['items_distribution'],
        "items_attack": res1['item_attacks_all']
    }

for a_pos in range(upgrades):
    scroll_list = ["B", "B", "B", "B", "B", "B", "B"]
    scroll_list[a_pos] = "A"
    strat1 = strategy_fixed_sequence(scroll_list)
    res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

    small_dict = [i['B'] for i in res1["scrolls_distribution_by_type"]]
    big_dist = [i['A'] for i in res1["scrolls_distribution_by_type"]]
    results["-".join(scroll_list)] = {
        SCROLL_SET["B"].name: np.mean(small_dict), 
        SCROLL_SET["A"].name: np.mean(big_dist), 
        "items": res1['items_distribution'],
        "items_attack": res1['item_attacks_all']
    }

In [ ]:
GROVE_ATTACK_PRICE = {
    10: [1500, 1488, 1499, 1488, 1488, 1450, 1450, 1455, 1399, 1350, 1499, 1288, 1499, 1338, 1335, 1234, 999, 1500],
    11: [2500, 2498, 2500, 2423, 2421, 2444, 2333, 2500],
    12: [5222, 5333, 5333, 5444, 5333, 5111, 5100, 5000, 5400, 5188, 5180, 5200, 5188, 5100, 5150, 5388, 5222, 5255, 5111],
    13: [13876, 15555, 15000, 15000, 14888, 14777, 14655, 14444, 14222, 11111],
    14: [40888, 41000, 40560],
    15: [122706]
}

def get_bad_grove_price(item_attacks_all):
    sell_item = []
    for attack_list in item_attacks_all:
        attack_array = np.array(attack_list)
        v,c = np.unique(attack_array[attack_array>=10], return_counts=True)

        bad_item_price = []
        for i,v_i in enumerate(v):
            bad_item_price.append(GROVE_ATTACK_PRICE[v_i] * c[i])

        sell_item.append(bad_item_price)
    return sell_item

In [ ]:
sell_items = get_bad_grove_price(result_i['items_attack'])

In [ ]:
len(sell_items[0])

In [ ]:
GROVE_SCROLL_PRICE = {"A": 33, "B": 597, "C": 14959, "D": 2325}
item_price = 2500
mxb_to_rmb = 55 # 55W mxb = 1rmb

first = True
num_scrolls = {}
for scroll_list, result_i in results.items():

    total_price = 0
    bad_grove_price = 0
    for key, value in result_i.items():
        if key == "items":
            total_price += np.mean(np.array(value) * item_price)
        elif key == "items_attack":
            bad_grove_price = np.mean(get_bad_grove_price(value))
        else:
            scroll_price = value * GROVE_SCROLL_PRICE[NAME_TO_KEY[key]]
            total_price += scroll_price
    
    bad_grove_sell = bad_grove_price / mxb_to_rmb
    cost_rmb = total_price / mxb_to_rmb
    if first:
        print(f"{scroll_list}: {cost_rmb:.2f} bad grove {bad_grove_sell:.2f} total cost {(cost_rmb-bad_grove_sell):.2f} {GROVE_SCROLL_PRICE}")
        first = False
    else:
        print(f"{scroll_list}: {cost_rmb:.2f} bad grove {bad_grove_sell:.2f} total cost {(cost_rmb-bad_grove_sell):.2f}")

## 2张30%

In [ ]:
rng = default_rng(100)
upgrades = 7
trials = 20000

# 目标：全成功
target = stop_atk_at_least(20)

# 策略
results = {}
for c_pos in range(upgrades):
    for c2_pos in range(upgrades):
        if c_pos == c2_pos:
            continue

        scroll_list = ["D", "D", "D", "D", "D", "D", "D"]
        scroll_list[c_pos] = "C"
        scroll_list[c2_pos] = "C"

        strat1 = strategy_fixed_sequence(scroll_list)
        res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

        small_dict = [i['D'] for i in res1["scrolls_distribution_by_type"]]
        big_dist = [i['C'] for i in res1["scrolls_distribution_by_type"]]
        results["-".join(scroll_list)] = {
            SCROLL_SET["D"].name: np.mean(small_dict), 
            SCROLL_SET["C"].name: np.mean(big_dist), 
            "items": res1['avg_items'],
            "items_attack": res1['item_attacks_all']
        }

weapon_price_by_attack = {"116": 1000, "117": 2500, "118": 5666, "119": 13333, "120": 23333, "121": 30000, "122": 40000, 
                          "123": 60000, "124": 70000, "125": 75000, "126": 88875, "127": 100000, "128": 120000}

item_price = WEAPON_SWORD['108']

first = True
for scroll_list, result_i in results.items():

    total_price = 0
    num_scrolls = {}
    for key, value in result_i.items():
        if key == "items":
            num_items = value
            total_price += value * item_price
        elif key == "items_attack":
            bad_items = []
            for i in result_i['items_attack']:
                bad_items.extend(i)

            num_bad_items = len(bad_items)/len(result_i['items_attack'])
            bad_weapon_rmb = np.mean(get_bad_weapon_rmb(value))
        else:
            num_scrolls[NAME_TO_KEY[key]] = value
            scroll_price = value * WEAPON_SCROLL_PRICE[NAME_TO_KEY[key]]
            total_price += scroll_price
    
    rmb = total_price / 52
    if first:
        print(f"{scroll_list}: {rmb:.2f} bad weapon {bad_weapon_rmb:.2f} total cost {rmb-bad_weapon_rmb:.2f} {WEAPON_SCROLL_PRICE} {WEAPON_SWORD}")
        print(f"items number: {num_items}, scrolls number: {num_scrolls}, bad items number: {num_bad_items}")
        first = False
    else:
        print(f"{scroll_list}: {rmb:.2f} bad weapon {bad_weapon_rmb:.2f} total cost {rmb-bad_weapon_rmb:.2f}")
        print(f"items number: {num_items}, scrolls number: {num_scrolls}, bad items number: {num_bad_items}")

## 3张30%

In [ ]:
from tqdm import tqdm

rng = default_rng(100)
upgrades = 7
trials = 20000

# 目标：全成功
target = stop_atk_at_least(23)

# 策略
results = {}
for c_pos in tqdm(range(upgrades)):
    for c2_pos in range(upgrades):
        for c3_pos in range(upgrades):
            if c_pos == c2_pos:
                continue
            if c2_pos == c3_pos:
                continue
            if c_pos == c3_pos:
                continue

            scroll_list = ["D", "D", "D", "D", "D", "D", "D"]
            scroll_list[c_pos] = "C"
            scroll_list[c2_pos] = "C"
            scroll_list[c3_pos] = "C"

            strat1 = strategy_fixed_sequence(scroll_list)
            res1 = monte_carlo_mix(rng, trials, strat1, upgrades, target)

            small_dict = [i['D'] for i in res1["scrolls_distribution_by_type"]]
            big_dist = [i['C'] for i in res1["scrolls_distribution_by_type"]]
            results["-".join(scroll_list)] = {
                SCROLL_SET["D"].name: np.mean(small_dict), 
                SCROLL_SET["C"].name: np.mean(big_dist), 
                "items": res1['avg_items'],
                "items_attack": res1['item_attacks_all']
            }

weapon_price_by_attack = {"116": 1000, "117": 2500, "118": 5666, "119": 13333, "120": 23333, "121": 30000, "122": 40000, 
                          "123": 60000, "124": 70000, "125": 75000, 
                          "126": 88875, "127": 100000, "128": 120000, 
                          "129": 140000, "130": 160000}

item_price = WEAPON_SWORD['108']

first = True
for scroll_list, result_i in results.items():

    total_price = 0
    num_scrolls = {}
    for key, value in result_i.items():
        if key == "items":
            num_items = value
            total_price += value * item_price
        elif key == "items_attack":
            bad_items = []
            for i in result_i['items_attack']:
                bad_items.extend(i)

            num_bad_items = len(bad_items)/len(result_i['items_attack'])
            bad_weapon_rmb = np.mean(get_bad_weapon_rmb(value))
        else:
            num_scrolls[NAME_TO_KEY[key]] = value
            scroll_price = value * WEAPON_SCROLL_PRICE[NAME_TO_KEY[key]]
            total_price += scroll_price
    
    rmb = total_price / 52
    if first:
        print(f"{scroll_list}: {rmb:.2f} bad weapon {bad_weapon_rmb:.2f} total cost {rmb-bad_weapon_rmb:.2f} {WEAPON_SCROLL_PRICE} {WEAPON_SWORD}")
        print(f"items number: {num_items}, scrolls number: {num_scrolls}, bad items number: {num_bad_items}")
        first = False
    else:
        print(f"{scroll_list}: {rmb:.2f} bad weapon {bad_weapon_rmb:.2f} total cost {rmb-bad_weapon_rmb:.2f}")
        print(f"items number: {num_items}, scrolls number: {num_scrolls}, bad items number: {num_bad_items}")